In [1]:
import itertools
import numpy as np

# Functions

## Functions for finding the basis of $\operatorname{gr}^{j} \bigwedge^m \mathfrak{g}$

In [2]:
# Example of initialization for I in SL_3(Z_p):

# e_sl3 = np.array([1,2,3,4,5,6,7,8], dtype=np.int32)
# e_sort_3 = np.array([0,3,4,6,7,1,2,5], dtype=np.int32)
# # 1-->0, 2-->3, 3-->4, 4-->6, 5-->7, 6-->1, 7-->2, 8-->5


# NOTE: Throughout, we use a tuple to represent a wedge product.



def non_zero_wedge(tup, g):
    """
    Return boolean describing whether tup is a non-zero wedge.

    Input:
    tup = a tuple of integers,
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    Boolean

    Description:
    True if tup contains more k's than dim(g_k)
    for any k from range(1, len(g)+1),
    where g_k = g[k-1],
    which implies that the wedge product is zero.
    False otherwise.
    """
    return all(tup.count(k) <= len(g[k - 1]) for k in range(1, len(g) + 1))


def grj_nwedge_g(j, n, g):
    """
    Return gradings for grade j of n wedges of g.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    Returns an np.array of lists of which n wedges gives grading j,
    each inner list correspond to a direct summand.

    Description:
    Goes through all ordered combinations (with repeats)
    of n numbers from 1 to len(g),
    and checks whether they give a non-zero wedge,
    and whether they have grading j alltogether.
    Also, ignores trivial cases.
    """
    if n <= 0 or j < n:
        return []

    base_gradings = [
        i
        for i in itertools.combinations_with_replacement(range(1, len(g) + 1), n)
        if non_zero_wedge(i, g) and sum(i) == j
    ]
    return base_gradings


def print_grj_nwedge_g(jmax, nmax, g):
    """
    Print all gradings of all wedges of g.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    No output, just prints gradings.

    Description:
    Goes through j from 0 to jmax and n from 0 to nmax,
    and check if grade j of n wedges of g is non-zero,
    and if so prints it.
    """
    print(
        """
        ===================================================
        gr^j of n wedges of g in terms of g^1, g^2 and g^3:
        =================================================== \n
        """
    )
    print(
        """
        Note:
        - This prints [(1)] as [(1,)].
        - A tuple corresponds to a wedge product,
          e.g., (1,1,2) mean g^1 wedge g^1 wedge g^2.
        - Tuples in the list correcsponds to summands,
          e.g., [(1,3),(2,2)] means
          g^1 wedge g^3 direct sum g^2 wedge g^2.
        - This doesn't work for 0 wedges of g nor for
          dim(g) wedges of g. \n
        """
    )
    for j in range(jmax + 1):
        print("------------")
        print(f"Grade {j}: \n")
        for n in range(nmax):
            grj_nwedge_g_tmp = grj_nwedge_g(j, n, g)
            if len(grj_nwedge_g_tmp) != 0:
                print(grj_nwedge_g_tmp)


def flatten_basis(basis):
    """
    Flattens a list of lists.

    Input:
    A basis in the form of
    np.array([[basis-elements-1],[basis-elements-2],...]),
    where each inner list corresponds to a direct summand.

    Output:
    A flat basis in the form
    np.array([basis-elements-1,basis-elements-2,...])

    Description:
    Flattens bases of direct sums to a list of all basis elements
    without separate lists for the direct summands.
    """
    return [item for direct_summand in basis for item in direct_summand]


def grj_nwedge_g_basis(j, n, g):
    """
    Return basis for grade j of n wedges of g.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    Returns an np.array of lists of basis elements for
    n wedges of g with grading j.
    Each inner list correspond to a direct summand.

    Description:
    Goes through all gradings (grade j with n wedges),
    counts the number of g_i for each i,
    finds basis for wedges of g_i for each i,
    puts the above together to a basis of
    grade j of n wedges of g.
    Finally, prettify the basis.
    """
    base_gradings = grj_nwedge_g(j, n, g)
    basis = []
    for grading in base_gradings:
        grade_count = [grading.count(i) for i in range(1, len(g) + 1)]
        basis_grading_indices = [
            tuple(itertools.combinations(range(len(g[i])), grade_count[i]))
            for i in range(len(g))
        ]
        grading_basis_ugly = itertools.product(*basis_grading_indices)
        grading_basis_pretty = []
        for base in grading_basis_ugly:
            pretty_base_element = []
            for k in range(len(g)):
                for index in base[k]:
                    pretty_base_element.append(g[k][index])
            grading_basis_pretty.append(tuple(pretty_base_element))
        basis.append(grading_basis_pretty)
    return np.array(flatten_basis(basis), dtype=np.int32)


def store_bases(jmax, nmax, g):
    """
    Store the all bases in an np.array.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output: 
    An np.array with a basis for grade j of n wedges of g
    in entry array[j][n].

    Description:
    Goes through j from 0 to jmax+2 and n from 0 to nmax+2,
    and saves the bases of grade j of n wedges of g. We go
    +2 because we want bases_array[jmax+1][nmax+1] to work
    for later use.
    """
    bases_array = np.empty((jmax+2, nmax+2), dtype=object)
    for j in range(jmax+2):
        for n in range(nmax+2):
            bases_array[j][n] = grj_nwedge_g_basis(j, n, g)
    return bases_array


## Functions for finding matrices of differential maps in the (co-)chain complex

Note that when going from the chain complex map $d$ described in a basis to the co-chain complex map in the dual basis, we are doing a matrix transpose (so it becomes $d^{\top}$), but we don't do this in the following, since we are actually finding the transpose matrix directly.

In [21]:
# Make sure to implement commutator(a,b) in each example.
# E.g. for I in SL_3(Z_p):
# def commutator(a, b):
# sign = 1
# if a > b:
#     a_tmp = a
#     a = b
#     b = a_tmp
#     sign = -1
# # to make it easier later, we return lists with the answers, where a x+y = [x,y]
# if a==1 and b==6:
#     return [(-sign,2)]
# elif a==1 and b==7:
#     return [(sign,3)]
# elif a==1 and b==8:
#     return [(-sign,5),(-sign,4)] # for the sake of later code, we order these in reverse order
# elif a==2 and b==7:
#     return [(-sign,4)]
# elif a==3 and b==6:
#     return [(-sign,5)]
# elif a==6 and b==7:
#     return [(-sign,8)]
# else:
#     return [(0,0)]

# Here we use a list to represent a sum, which will work with our implementations.


def fast_base_sort(g, eis, e_sort):
    """
    Sort our basis to the order we prefer.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements) out of order
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's

    Output:
    A tuple of the e_i's sorted.

    Description:
    We use that, when constructing the e_i's in the following, 
    we can only really mess up the order of one element, 
    so we just linearly check the order of the elements, 
    and compare to e_sort, remembering to change signs, 
    when moving elements past wedges.
    
    Recall that a tuple corresponds to a wedge product.
    """
    eis = list(eis)
    sign = 1
    # Don't look at eis[0], since that is just the coef
    for j in range(1, len(eis) - 1):
        if e_sort[eis[j] - 1] > e_sort[eis[j + 1] - 1]:
            eis[j], eis[j + 1] = eis[j + 1], eis[j]
            # Equivalent to:
            # tmp = eis[j]
            # eis[j] = eis[j+1]
            # eis[j+1] = tmp
            sign *= -1
    # Put coefficient on first entry
    eis[0] = sign * eis[0]
    return tuple(eis)


def d(g, eis, e_sort, commutator):
    """
    Calculate the d of eis in the (co-)chain complex.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements)
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's
    commutator = a function (int,int) -> [(int (coefficient),int (results))],
    where different elements of the list corespond to summands.

    Output:
    A list of tuples of the e_i's sorted.
    Elements of the list correspond to summands.
    We put the coefficient on the first element of each tuple.

    Description:
    Calculate d(eis) for the wedge product eis of e_i's,
    while making sure to place the coefficient in the first entry,
    and keeping the e_i's in our desired order.
    
    NOTE: Remember to implement commutator(a, b) first.
    """
    d_eis = []
    eis = np.array(eis)
    d_eis_elem = np.zeros(len(eis) - 1, dtype=np.int32)
    for j in range(2, len(eis)):
        for i in range(1, j):
            com = commutator(eis[i], eis[j])
            # Use that we put com[0][0] = 0, if the commutator is 0.
            if com[0][0] != 0:
                sign = (-1) ** (i + j)
                d_eis_elem_tmp = [
                    eis[k] for k in range(1, len(eis)) if k != i and k != j
                ]
                # Check if wedge product should be zero, because of repeat e_i
                for k in range(len(com)):
                    if not com[k][1] in d_eis_elem_tmp:
                        d_eis_elem[0] = sign * com[k][0] * eis[0]
                        d_eis_elem[1] = com[k][1]
                        d_eis_elem[2:] = d_eis_elem_tmp
                        d_eis.append(fast_base_sort(g, d_eis_elem, e_sort))
    return d_eis


def add_one_coef(arr):
    """
    Add one (first) entry to arr with 1.

    Input:
    An np.array of size n.

    Output:
    An np.array of size n+1 with 1 in the first entry.

    Description:
    Construct a new array of size n+1 with all 1's.
    Change the last n entries to equal arr.
    """
    arr_with_one = np.ones(len(arr) + 1, dtype=np.int32)
    arr_with_one[1:] = arr
    return arr_with_one


def d_coefs(g, eis, e_sort, codomain_basis, commutator):
    """
    Calculate the coefficients describing the map d in the chain complex.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements)
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's,
    codomain_basis = basis of codomain of d.
    commutator = a function (int,int) -> [(int (coefficient),int (results))],
    where different elements of the list corespond to summands.

    Output:
    An np.array of coefficients for d(eis) in the basis of
    the codomain given by basis in the input.

    Description:
    Return np.array([0]) if d(eis) = 0 (trivially).
    Otherwise, calculate d(eis), and find the coefficients
    of d(eis) in codomain_basis.
    """
    # len(eis) <= 2 instead of 1, since the first entry is just the coefficient
    if len(codomain_basis) == 0 or len(eis) <= 2:
        return np.array([0], dtype=np.int32)
    d_eis = d(g, eis, e_sort, commutator)
    coefs = np.zeros(len(codomain_basis), dtype=np.int32)
    for d_ei in d_eis:
        find_in_basis = np.all(codomain_basis == np.array(d_ei[1:]), axis=1)
        coef_index = np.where(find_in_basis)
        coefs[coef_index] = d_ei[0]
    return coefs

def d_matrix_grj_nwedge_g(j, n, g, e_sort, bases, commutator):
    """
    Return transpose of matrix d out from grade j of n wedges of g.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    bases = np.array with all bases in any grade and any wedges.
    commutator = a function (int,int) -> [(int (coefficient),int (results))],
    where different elements of the list corespond to summands.

    Output:
    Returns an np.array for the transpose matrix of the map d from
    grade j of n wedges of g to grade j of n-1 wedges of g, or
    equivalently,
    the matrix describing the map d^T
    from grade -j of Hom(n-1 wedges of g, k)
    to grade -j of Hom(n wedges of g, k).

    Description:
    First compute the bases of the codomain and domain of d,
    then (unless trivial) find coefficients to represent the
    image in the codomain basis. Finally, note that we return
    the transpose, since we fill in rows instead of columns,
    which is the map we actually care about anyways.
    """
    codomain_basis = bases[j][n - 1]
    domain_basis = bases[j][n]
    if len(codomain_basis) == 0 or len(domain_basis) == 0:
        return np.array([0], dtype=np.int32)
    # Note, that our basis doesn't contain coefficients,
    # so we add the coefficient 1 to eis
    matrix = [
        d_coefs(g, add_one_coef(eis), e_sort, codomain_basis, commutator)
        for eis in domain_basis
        if len(eis) > 1
    ]
    return np.array(matrix, dtype=np.int32)

def store_d_matrices(jmax, nmax, g, e_sort, bases, commutator):
    """
    Store the all the matrices describing d in an np.array.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    bases = np.array with all bases in any grade any wedges.
    commutator = a function (int,int) -> [(int (coefficient),int (results))],
    where different elements of the list corespond to summands.

    Output:
    An np.array with a matrix to the map
    gr^(-j) Hom(n-1 wedges of g, k) --> Hom(n wedges of g, k)
    in entry array[j][n].

    Description:
    Goes through j from 0 to jmax+1 and n from 0 to nmax+1,
    and saves the matrix d to grade -j of Hom(n wedges of g, k).
    We go to +1 to have the entry d_matrices_array[jmax+1][nmax+1]
    make sense.
    """
    d_matrices_array = np.empty((jmax+1, nmax+1), dtype=object)
    for j in range(jmax+1):
        for n in range(nmax+1):
            d_matrices_array[j][n] = np.array([0], dtype=np.int32)
    for j in range(1, jmax):
        for n in range(1, nmax):
            d_matrices_array[j][n] = d_matrix_grj_nwedge_g(
                j, n, g, e_sort, bases, commutator
            )
    return d_matrices_array


def print_cochain_matrices(d_matrices):
    """
    Print all non-trivial cochain matrices.

    Input:
    d_matrices = np.array with matrices,
    where the matrices describe maps
    gr^(-j) Hom(n-1 wedges of g, k) --> gr^(-j) Hom(n wedges of g, k).

    Output:
    No output, but prints matrices.

    Description:
    Print (nicely) the matrices, 
    plus some useful information.
    """
    jmax1, nmax1 = d_matrices.shape
    print(
        """
        ===================================================
        Non trivial matrices:
        =================================================== \n
        """
    )
    for j in range(jmax1):
        for n in range(nmax1):
            s = -j
            t = n - s
            # we have the dual map of (j,n) --> (j,n-1), i.e., (j,n-1) --> (j,n)
            if np.count_nonzero(d_matrices[j][n]) > 0:
                print(f"s = {s}, t = {t}, s+t = {s+t} ; (j = {j}, n = {n}):")
                print(
                    f"""
                    d : Hom^({s})({s+t-1} wedges of g, k) --> Hom^({s})({s+t} wedges of g, k)
                    """
                )
                print(f"matrix shape: {d_matrices[j][n].shape}")
                print("\n Matrix:\n")
                # NOTE: display/show works in jupyter notebooks instead of print,
                # use print to get matrices that look worse but also take up less
                # space
                show(matrix(d_matrices[j][n], ring=GF(5)))
                print("\n LaTeX code:")
                print("Matrix: " + latex(matrix(d_matrices[j][n], ring=GF(5))))
                print("\n ------------------------------------------- \n\n")


## Functions for finding bases of the kernel and image of the differential maps

In [23]:
def kernel_and_image_bases_j_n(j, n, bases, d_matrices):
    """
    Return a basis of the null and column space of the matrix
    corresponding to the map
    gr^(-j) Hom(n-1 wedges of g, k) --> gr^(-j) Hom(n wedges of g, k).

    Input:
    j = grading (≥1),
    n = number of wedges (≥1, < dim(g)),
    bases = np.array with all bases in any grade any wedges,
    d_matrices = np.array with matrices,
    where the matrices describe maps
    gr^(-j) Hom(n-1 wedges of g, k) --> gr^(-j) Hom(n wedges of g, k).

    Output:
    Returns two lists of bases.

    Description:
    If we map to or from a space of dimension 0, we handle that
    separately, and if that is not the case, we have a matrix
    that we can work with and find the null/column space.
    """
    if n == 0:
        return [], []
    codomain_basis = bases[j][n]
    codomain_dim = len(codomain_basis)
    domain_basis = bases[j][n - 1]
    domain_dim = len(domain_basis)
    if codomain_dim == 0 or domain_dim == 0:
        this_d = MatrixSpace(GF(5), codomain_dim, domain_dim)(0)
        this_image_basis = this_d.column_space().basis()
        this_kernel_basis = this_d.right_kernel().basis()
        return this_kernel_basis, this_image_basis
    this_d = Matrix(d_matrices[j][n], ring=GF(5))
    this_image_basis = this_d.column_space().basis()
    this_kernel_basis = this_d.right_kernel().basis()
    return this_kernel_basis, this_image_basis


def store_ker_im_bases(bases, d_matrices):
    """
    Store all the kernel and image bases for the different d
    in an np.array.

    Input:
    bases = np.array with all bases in any grade any wedges,
    d_matrices = np.array with matrices,
    where the matrices describe maps
    gr^(-j) Hom(n-1 wedges of g, k) --> gr^(-j) Hom(n wedges of g, k).

    Output:
    An np.array with a list of kernel and image bases for each entry.

    Description:
    Goes through j from 0 to jmax+1 and n from 0 to nmax+1,
    and saves the basis corresponding to the matrix d 
    to grade -j of Hom(n wedges of g, k).
    We go to +1 to have the entry d_matrices_array[jmax+1][nmax+1]
    make sense.
    """
    (jmax2, nmax2) = bases.shape
    ker_im_bases = np.empty((jmax2, nmax2), dtype=object)
    for j in range(jmax2):
        for n in range(nmax2):
            ker_im_bases[j][n] = kernel_and_image_bases_j_n(j, n, bases, d_matrices)
    return ker_im_bases


In [ ]:



def print_ker_im_bases(ker_im_bases):
    """
    Print all non-trivial kernel/image bases.

    Input:
    ker_im_bases = np.array containing bases of the kernel
    and the image of the different d.

    Output:
    No output, but prints bases.

    Description:
    Print (nicely) the bases, 
    plus some useful information.
    """
    jmax2, nmax2 = ker_im_bases.shape
    print(
        """
        ===================================================
        Non trivial kernel and image bases:
        =================================================== \n
        """
    )
    for j in range(jmax2):
        for n in range(nmax2):
            s = -j
            t = n - s
            # we have the dual map of (j,n) --> (j,n-1), i.e., (j,n-1) --> (j,n)
            if len(ker_im_bases[j][n][0]) > 0 or len(ker_im_bases[j][n][1]) > 0:
                print(f"s = {s}, t = {t}, s+t = {s+t} ; (j = {j}, n = {n}):")
                print(
                    f"""
                    d : Hom^({s})({s+t-1} wedges of g, k) --> Hom^({s})({s+t} wedges of g, k)
                    """
                )
                if len(ker_im_bases[j][n][0]) > 0:
                    print(f"kernel basis:\n    {ker_im_bases[j][n][0]}")
                if len(ker_im_bases[j][n][1]) > 0:
                    print(f"image basis:\n    {ker_im_bases[j][n][1]}")
                print("\n ------------------------------------------- \n\n")


## Functions for finding bases of $H^{s,t}$ for each $s,t$

In [39]:
def H_s_t(j, n, jmax, nmax, ker_im_bases):
    """
    Return the spaces of the kernel, image, and
    kernel/image (i.e. cohomology) corresponding
    to the map d to gr^(-j) Hom(n wedges of g, k).

    Input:
    j = grading (≥1),
    n = number of wedges (≥1, < dim(g)),
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    ker_im_bases = np.array containing bases of the kernel
    and the image of the different d.

    Output:
    Returns three spaces: kernel, image, kernel/image.

    Description:
    In trivial cases (n=j=0 and j=jmax,n=nmax) we hard-code
    the corresponding spaces. In other cases, we use the
    kernel and image bases to find the kernel and image
    spaces by looking at the span of the bases. We then
    find kernel/image using standard Sage functions. 
    """    
    # s = -j
    # t = n+j
    triv_mod = FreeModule(GF(5), 1)
    zero_mod = FreeModule(GF(5), 0)
    if j == 0 and n == 0:
        return triv_mod, zero_mod, triv_mod/zero_mod
    if j == jmax and n == nmax:
        return triv_mod, zero_mod, triv_mod/zero_mod
    this_d_ker_basis, this_d_im_basis = ker_im_bases[j][n]
    next_d_ker_basis, next_d_im_basis = ker_im_bases[j][n+1]
    if len(next_d_ker_basis) > 0:
        this_dim = len(next_d_ker_basis[0])
    elif len(this_d_im_basis) > 0:
        this_dim = len(this_d_im_basis[0])
    else:
        return zero_mod, zero_mod, zero_mod/zero_mod
    M = FreeModule(GF(5), this_dim) # ambient module
    ker = M.span(next_d_ker_basis, GF(5))
    im = M.span(this_d_im_basis, GF(5))
    # the image should always be a submodule of the kernel
    if not im.is_submodule(ker):
        print("ERROR!")
    return ker, im, ker/im

def store_H_s_t(ker_im_bases):
    """
    Store all the H^(s,t) spaces in an np.array.

    Input:
    ker_im_bases = np.array containing bases of the image
    and the kernel of the different d.

    Output:
    An np.array with a H^(s,t) space for each entry.

    Description:
    Goes through j from 0 to jmax+1 and n from 0 to nmax+1,
    and saves the H^(s,t) space in an np.array.
    """
    jmax2, nmax2 = ker_im_bases.shape
    H_s_t_array = np.empty((jmax2-1,nmax2-1), dtype=object)
    zero_mod = FreeModule(GF(5), 0)
    for j in range(jmax2-1):
        for n in range(nmax2-1):
            _, _, Hst = H_s_t(j, n, jmax2-2, nmax2-2, ker_im_bases)
            H_s_t_array[j][n] = Hst
    return H_s_t_array



In [7]:

def print_H_s_t(H_s_t_array):
    """
    Print all H^(s,t) spaces from the np.array.

    Input:
    H_s_t_array = np.array containing cohomology
    spaces H^(s,t) (either with or without 2-torsion).

    Output:
    No output, but prints spaces.

    Description:
    Print (nicely) the spaces, 
    plus some useful information.
    """
    jmax1, nmax1 = H_s_t_array.shape
    print(
        """
        ===================================================
        Non trivial cohomology H^(s,t) spaces:
        =================================================== \n
        """
    )
    for j in range(jmax1):
        for n in range(nmax1):
            s = -j
            t = n - s
            # we have the dual map of (j,n) --> (j,n-1), i.e., (j,n-1) --> (j,n)
            Hst = H_s_t_array[j][n]
            if Hst.ngens() > 0:
                print(f"(s,t) = ({s},{j}): dim H^({s},{t}) = {Hst.ngens()}\n")
                print(f"H^({s},{t}) = {Hst}\n---\n")


In [67]:


def print_H_s_t_bases(H_s_t_array):
    """
    Print bases for all H^(s,t) spaces from the np.array.

    Input:
    H_s_t_array = np.array containing cohomology
    spaces H^(s,t) (either with or without 2-torsion).

    Output:
    No output, but prints bases of spaces.

    Description:
    Print (nicely) the bases of the spaces, both in the space
    itself and the lift to a basis of the corresponding ambient
    space, plus some useful information.
    """
    jmax1, nmax1 = H_s_t_array.shape
    print(
        """
        ===================================================
        Bases of non trivial cohomology H^(s,t) spaces:
        =================================================== \n
        """
    )
    for j in range(jmax1):
        for n in range(nmax1):
            s = -j
            t = n - s
            # we have the dual map of (j,n) --> (j,n-1), i.e., (j,n-1) --> (j,n)
            Hst = H_s_t_array[j][n]
            if Hst.ngens() > 0:
                print(f"(s,t) = ({s},{j}): dim H^({s},{t}) = {Hst.ngens()}\n")
                print(f"basis in H^({s},{t}):\n {Hst.gens()}\n")
                print(f"lift of basis to the ambient space:\n {list(map(Hst.lift, Hst.gens()))}\n---\n")


## Functions for finding the cup product

In [90]:
def non_zero_tuple(tup):
    """
    Return boolean describing whether tup is a non-zero wedge.

    Input:
    tup = a tuple of integers

    Output:
    Boolean

    Description:
    True if tup contains distinct integers.
    False otherwise.
    """
    return (len(tup) == len(set(tup)))


def slow_base_sort(g, eis, e_sort):
    """
    Sort our basis to the order we prefer.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements) out of order
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's

    Output:
    A tuple of the e_i's sorted.

    Description:
    Compare the order to e_sort, and change it until it matches,
    remembering to change signs, when moving elements past wedges.
    Recall that a tuple corresponds to a wedge product.
    """
    eis = list(eis)
    sign = 1
    for i in range(1, len(eis) - 1):
        for j in range(1,len(eis)-1):
            if e_sort[eis[j] - 1] > e_sort[eis[j + 1] - 1]:
                eis[j], eis[j + 1] = eis[j + 1], eis[j]
                # Equivalent to:
                # tmp = eis[j]
                # eis[j] = eis[j+1]
                # eis[j+1] = tmp
                sign *= -1
    # Put coefficient on first entry
    eis[0] = sign * eis[0]
    return tuple(eis)


def hom_cup_product(b1, b2, g, e_sort):
    """
    Find the cup product of b1 and b2: b1 u b2.
    
    Input:
    b1 = np.array([e1,e2,...]) a basis vector (assume sorted)
    b2 = np.array([e1',e2',...]) a basis vector (assume sorted)
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's
    
    Output:
    The cup product b1 u b2 in the hom-space (not in cohomology).
    
    Description:
    Find the coefficient, if the tuple corresponds to a zero 
    wedge (i.e., if it has the same element twice) return (0,),
    otherwise resorting the indices and keeping track of the 
    sign will give the correct cupproduct, since 
    we are assuming that we are only working with a
    basis and its dual basis all the the time.
    """
    b1_tmp = tuple(b1[1:]) # b1 without the coef
    b2_tmp = tuple(b2[1:]) # b2 without the coef
    coef = b1[0] * b2[0]
    #tmp_cup = np.concatenate(b1_tmp, b2_tmp)
    tmp_cup = b1_tmp + b2_tmp # concatenate tuples
    if not (non_zero_tuple(tmp_cup)):
        return (0,) #tuple with coef = 0
    cup_with_coef = slow_base_sort(g, (coef,) + tmp_cup, e_sort)
    return cup_with_coef


def vec_to_basis(j, n, vec, bases):
    """
    Given a vector describing coeffients in a basis,
    return an np.array with the corresponding basis
    elements and their coefficient.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1, < dim(g)),
    vec = a vector containing coefficients for each of the
    corresponding basis elements from bases,
    bases = np.array with all bases in any grade any wedges.

    Output:
    Returns an np.array with basis elements with a coefficient.

    Description:
    We look at the basis for the given j and n. If it is empty,
    we return an empty list (when using this function, we should
    not get to this point). Otherwise make an np.array to contain
    all basis elements with a non-zero coefficient in the vector,
    and add the basis elements with a coefficient to this array.
    """
    basis = bases[j][n]
    if len(basis) == 0:
        return []
    num_basis, num_wedges = basis.shape
    non_zero_vec = np.count_nonzero(vec)
    vec_basis = np.zeros((non_zero_vec, num_wedges+1), dtype=np.int32)
    j = 0
    for i in range(num_basis):
        if vec[i] != 0:
            coef = vec[i]
            vec_basis[j][0] = coef
            vec_basis[j][1:] = basis[i]
            j += 1
    return vec_basis


def Hs1t1_Hs2t2_cups(j1, n1, j2, n2, g, e_sort, H_s_t_array, bases):
    """
    Given H^(s1,t1) and H^(s2,t2), we calculate all cup
    products (under certain assumptions).

    Input:
    j1 = grading (≥1),
    n1 = number of wedges (≥1, < dim(g)),
    j2 = grading (≥1),
    n2 = number of wedges (≥1, < dim(g)),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    H_s_t_array = np.array containing cohomology
    spaces H^(s,t) (either with or without 2-torsion),
    bases = np.array with all bases in any grade any wedges.

    Output:
    Returns an np.array with cup products.

    Description:
    We lift the H^(s1,t1) and H^(s2,t2) bases to their ambient 
    space, then we check for a lot of conditions that we assume
    are not satisfied here. When there is a possible non-zero
    cup product, we go through all the possible cases and 
    calculate all non-zero cup products. 
    """
    Hs1t1 = H_s_t_array[j1][n1]
    Hs1t1_basis = list(map(Hs1t1.lift, H_s_t_array[j1][n1].gens()))
    Hs1t1_dim = H_s_t_array[j1][n1].ngens()
    Hs2t2 = H_s_t_array[j2][n2]
    Hs2t2_basis = list(map(Hs2t2.lift, H_s_t_array[j2][n2].gens()))
    Hs2t2_dim = H_s_t_array[j2][n2].ngens()
    jmax1, nmax1 = H_s_t_array.shape
    if (j1+j2 >= jmax1) or (n1+n2 >= nmax1):
        # We assume we don't get to here
        print("ERROR 1")
        return []
    Hst_sum = H_s_t_array[j1+j2][n1+n2]
    Hst_sum_dim = Hst_sum.ngens()
    base_list = bases[j1+j2][n1+n2].tolist()
    Hst_sum_am_dim = len(Hst_sum.lift(Hst_sum.gen(0)))
    cups_array = np.empty((Hs1t1_dim, Hs2t2_dim), dtype=object)
    if Hs1t1_dim == 0:
        print("ERROR 2")
        return []
    if Hs2t2_dim == 0:
        print("ERROR 3")
        return []
    if Hst_sum_dim == 0:
        # WE assume we don't get to here
        print("ERROR 4")
        return []
    for i1 in range(Hs1t1_dim):
        vec1 = Hs1t1_basis[i1]
        vec1_in_basis = vec_to_basis(j1, n1, vec1, bases)
        for i2 in range(Hs2t2_dim):
            vec2 = Hs2t2_basis[i2]
            vec2_in_basis = vec_to_basis(j2, n2, vec2, bases)
            cup_sum_array = np.zeros(Hst_sum_am_dim, dtype=np.int32)
            for b1 in vec1_in_basis:
                for b2 in vec2_in_basis:
                    hom_b1_b2_cup = hom_cup_product(b1, b2, g, e_sort)
                    if hom_b1_b2_cup[0] != 0:
                        coef = hom_b1_b2_cup[0]
                        hom_b1_b2_cup_list = list(hom_b1_b2_cup[1:])
                        tmp_b1_b2_cup = np.zeros(Hst_sum_am_dim, dtype=np.int32)
                        ind = base_list.index(hom_b1_b2_cup_list)
                        tmp_b1_b2_cup[ind] = coef
                        cup_sum_array += tmp_b1_b2_cup
            cup_sum = tuple(cup_sum_array.tolist())
            #cups_array[i1][i2] = Hst_sum(cup_sum)
            cups_array[i1][i2] = Hst_sum.quotient_map()(cup_sum)
    return cups_array


In [58]:
V = FreeModule(GF(5), 5)
tmpA = V.span([[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0]], GF(5))
tmpB = V.span([[1,0,0,0,0]], GF(5))
tmpC = tmpA/tmpB
tmp = tmpC.gen(0)
tmp.lift(), tmpC.lift(tmp), tmpC.gens()

((1, 0), (0, 1, 0, 0, 0), ((1, 0), (0, 1)))

In [60]:
list(map(tmpC.lift, tmpC.gens()))

[(0, 1, 0, 0, 0), (0, 0, 1, 0, 0)]

In [89]:
tmpC.quotient_map()((1,0,0,0,0))

(0, 0)

In [11]:


def print_cup_products(g, e_sort, H_s_t_array, bases):
    """
    Print all non-zero cup products with some extra
    information.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    H_s_t_array = np.array containing cohomology
    spaces H^(s,t) (either with or without 2-torsion),
    bases = np.array with all bases in any grade any wedges.

    Output:
    No output, but prints non-zero cup products.

    Description:
    Print (nicely) the cup products of the different H^(s,t)
    spsaces, plus some useful information.
    """
    jmax1, nmax1 = H_s_t_array.shape
    print(
        """
        ===================================================
        Non-zero cup products from H^(s,t) spaces:
        =================================================== \n
        """
    )
    # we are not looking at the cup product for H^0
    print("NOTE: We are not printing any H^(0) cup products.\n")
    print("We write vi^(s,t) for the i'th basis vector of H^(s,t).\n")
    for j1 in range(1, jmax1):
        for n1 in range(1, nmax1):
            Hst1 = H_s_t_array[j1][n1]
            Hst1_dim = Hst1.ngens()
            if Hst1_dim > 0:
                for j2 in range(j1, jmax1-j1):
                    for n2 in range(n1, nmax1-n1):
                        Hst2 = H_s_t_array[j2][n2]
                        Hst2_dim = Hst2.ngens()
                        Hst_sum = H_s_t_array[j1+j2][n1+n2]
                        Hst_sum_dim = Hst_sum.ngens()
                        if (Hst2_dim > 0) and (Hst_sum_dim > 0):
                            Hst_sum_cups = Hs1t1_Hs2t2_cups(j1, n1, j2, n2, g, e_sort, H_s_t_array, bases)
                            print("\n")
                            for i1 in range(Hst1_dim):
                                for i2 in range(Hst2_dim):
                                    if Hst_sum_cups[i1][i2] != Hst_sum(0):
                                        s1, t1, s2, t2 = -j1, n1+j1, -j2, n2+j2
                                        s3, t3 = s1+s2, t1+t2
                                        current_cup = np.array(list(Hst_sum_cups[i1][i2]), dtype=np.int32)
                                        cup_nonzero_index = np.nonzero(current_cup)
                                        current_cup_nonzero = current_cup[cup_nonzero_index]
                                        current_string = f"v{i1+1}^({s1},{t1}) cup v{i2+1}^({s2},{t2}) = "
                                        for ind in range(len(current_cup_nonzero)-1):
                                            current_string += f"{current_cup_nonzero[ind]}*v{cup_nonzero_index[0][ind]+1}^({s3},{t3}) + "
                                        current_string += f"{current_cup_nonzero[-1]}*v{cup_nonzero_index[0][-1]+1}^({s3},{t3})"
                                        print(current_string + "\n")
                                        #print(f"v{i1+1}^({-j1},{n1+j1}) cup v{i2+1}^({-j2},{n2+j2}) = {Hst_sum_cups[i1][i2]} in H^({-j1-j2},{n1+n2+j1+j2})\n")


In [12]:


def print_cup_products_alt(g, e_sort, H_s_t_array, bases):
    """
    Print all non-zero cup products with some extra
    information.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    H_s_t_array = np.array containing cohomology
    spaces H^(s,t) (either with or without 2-torsion),
    bases = np.array with all bases in any grade any wedges.

    Output:
    No output, but prints non-zero cup products.

    Description:
    Print (nicely) the cup products of the different H^(s,t)
    spsaces, plus some useful information.
    """
    jmax1, nmax1 = H_s_t_array.shape
    print(
        """
        ===================================================
        Non-zero cup products from H^(s,t) spaces:
        =================================================== \n
        """
    )
    # we are not looking at the cup product for H^0
    print("NOTE: We are not printing any H^(0) cup products.\n")
    print("We write vi^(s,t) for the i'th basis vector of H^(s,t).\n")
    for j1 in range(1, jmax1):
        for n1 in range(1, nmax1):
            Hst1 = H_s_t_array[j1][n1]
            Hst1_dim = Hst1.ngens()
            if Hst1_dim > 0:
                for j2 in range(j1, jmax1-j1):
                    for n2 in range(n1, nmax1-n1):
                        Hst2 = H_s_t_array[j2][n2]
                        Hst2_dim = Hst2.ngens()
                        Hst_sum = H_s_t_array[j1+j2][n1+n2]
                        Hst_sum_dim = Hst_sum.ngens()
                        if (Hst2_dim > 0) and (Hst_sum_dim > 0):
                            Hst_sum_cups = Hs1t1_Hs2t2_cups(j1, n1, j2, n2, g, e_sort, H_s_t_array, bases)
                            print("\n")
                            for i1 in range(Hst1_dim):
                                for i2 in range(Hst2_dim):
                                    if Hst_sum_cups[i1][i2] != Hst_sum(0):
                                        s1, t1, s2, t2 = -j1, n1+j1, -j2, n2+j2
                                        s3, t3 = s1+s2, t1+t2
                                        print(f"v{i1+1}^({s1},{t1}) cup v{i2+1}^({s2},{t2}) = {Hst_sum_cups[i1][i2]} in H^({s3},{t3})\n")


# Using the functions for $I_D = (1+\mathfrak{m}_D)^{\operatorname{Nrd}=1}$ when $D/\mathbb Q_5$ is dimension $9$ ($p=5$)

## Initialization

In [17]:
g_ID_1 = np.array([1,2,3], dtype=np.int32) #1-graded (1 = 1,2 = a,3 = a^2)
g_ID_2 = np.array([4,5,6], dtype=np.int32) #2-graded (4 = 1,5 = a,6 = a^2)
g_ID_3 = np.array([7,8], dtype=np.int32) #3-graded (7 = 1,8 = a^2)
g_ID = np.array([g_ID_1,g_ID_2,g_ID_3], dtype=object); 
g_ID

array([array([1, 2, 3], dtype=int32), array([4, 5, 6], dtype=int32),
       array([7, 8], dtype=int32)], dtype=object)

In [18]:
# g = g^1 + g^2 + g^3, we order them g^1, g^2, g^3, 
# where g^1 = span(e_1, e_2, e_3), g^2 = span(e_4, e_5, e_6), g^3 = span(e_7, e_8)
# so the prefered order is: 
# e_1 (1st), e_2 (2nd), e_3 (3rd), 
# e_4 (4th), e_5 (5th), e_6 (6th), 
# e_7 (7th), e_8 (8th)
# i.e., in a zero indexed list:
# (e_1 =) 1-->0, (e_2 =) 2-->1, 3-->2, 4-->3, 5-->4, 6-->5, 7-->6,, 8-->7
e_sort_ID = np.array([0,1,2,3,4,5,6,7], dtype=np.int32)

In [19]:
def commutator_ID(a, b):
    sign = 1
    if a > b:
        a,b = b,a
        sign = -1
    # to make it easier later, we return lists with the answers, where a x+y = [x,y]
    if a==1 and b==2:
        return [(2*sign,6),(3*sign,5),(4*sign,4)]
    elif a==1 and b==3:
        return [(4*sign,6),(2*sign,5),(3*sign,4)]
    elif a==1 and b==5:
        return [(sign,8),(3*sign,7)]
    elif a==1 and b==6:
        return [(2*sign,8),(2*sign,7)]
    elif a==2 and b==3:
        return [(4*sign,6),(sign,5),(2*sign,4)]
    elif a==2 and b==4:
        return [(sign,8),(sign,7)]
    elif a==2 and b==5:
        return [(2*sign,8),(sign,7)]
    elif a==2 and b==6:
        return [(2*sign,7)]
    elif a==3 and b==4:
        return [(sign,8),(2*sign,7)]
    elif a==3 and b==5:
        return [(3*sign,7)]
    elif a==3 and b==6:
        return [(2*sign,8)]
    else:
        return [(0,0)]

## Store relevant information

Note that $j_{\max} = 15$ and $n_{\max} = 8$ for $(1+\mathfrak{m}_D)^{\operatorname{Nrd} = 1}$ when $D/\mathbb Q_p$ is dimension $9$ ($p \geq 5$).

In [40]:
ID_jmax, ID_nmax = 15, 8
ID_bases = store_bases(ID_jmax, ID_nmax, g_ID)
ID_d_matrices = store_d_matrices(ID_jmax, ID_nmax, g_ID, e_sort_ID, ID_bases, commutator_ID)
ID_ker_im_bases = store_ker_im_bases(ID_bases, ID_d_matrices)
ID_H_s_t_array = store_H_s_t(ID_ker_im_bases)

## Print information

In [62]:
print_grj_nwedge_g(ID_jmax, ID_nmax, g_ID)


        gr^j of n wedges of g in terms of g^1, g^2 and g^3:

        

        Note:
        - This prints [(1)] as [(1,)].
        - A tuple corresponds to a wedge product,
          e.g., (1,1,2) mean g^1 wedge g^1 wedge g^2.
        - Tuples in the list correcsponds to summands,
          e.g., [(1,3),(2,2)] means
          g^1 wedge g^3 direct sum g^2 wedge g^2.
        - This doesn't work for 0 wedges of g nor for
          dim(g) wedges of g. 

        
------------
Grade 0: 

------------
Grade 1: 

[(1,)]
------------
Grade 2: 

[(2,)]
[(1, 1)]
------------
Grade 3: 

[(3,)]
[(1, 2)]
[(1, 1, 1)]
------------
Grade 4: 

[(1, 3), (2, 2)]
[(1, 1, 2)]
------------
Grade 5: 

[(2, 3)]
[(1, 1, 3), (1, 2, 2)]
[(1, 1, 1, 2)]
------------
Grade 6: 

[(3, 3)]
[(1, 2, 3), (2, 2, 2)]
[(1, 1, 1, 3), (1, 1, 2, 2)]
------------
Grade 7: 

[(1, 3, 3), (2, 2, 3)]
[(1, 1, 2, 3), (1, 2, 2, 2)]
[(1, 1, 1, 2, 2)]
------------
Grade 8: 

[(2, 3, 3)]
[(1, 1, 3, 3), (1, 2, 2, 3)]
[(1, 1, 1, 2, 3), (1

In [63]:
print_cochain_matrices(ID_d_matrices)


        Non trivial matrices:

        
s = -2, t = 4, s+t = 2 ; (j = 2, n = 2):

                    d : Hom^(-2)(1 wedges of g, k) --> Hom^(-2)(2 wedges of g, k)
                    
matrix shape: (3, 3)

 Matrix:



[1 2 3]
[2 3 1]
[3 4 1]


 LaTeX code:
Matrix: \left(\begin{array}{rrr}
1 & 2 & 3 \\
2 & 3 & 1 \\
3 & 4 & 1
\end{array}\right)

 ------------------------------------------- 


s = -3, t = 5, s+t = 2 ; (j = 3, n = 2):

                    d : Hom^(-3)(1 wedges of g, k) --> Hom^(-3)(2 wedges of g, k)
                    
matrix shape: (9, 2)

 Matrix:



[0 0]
[2 4]
[3 3]
[4 4]
[4 3]
[3 0]
[3 4]
[2 0]
[0 3]


 LaTeX code:
Matrix: \left(\begin{array}{rr}
0 & 0 \\
2 & 4 \\
3 & 3 \\
4 & 4 \\
4 & 3 \\
3 & 0 \\
3 & 4 \\
2 & 0 \\
0 & 3
\end{array}\right)

 ------------------------------------------- 


s = -3, t = 6, s+t = 3 ; (j = 3, n = 3):

                    d : Hom^(-3)(2 wedges of g, k) --> Hom^(-3)(3 wedges of g, k)
                    
matrix shape: (1, 9)

 Matrix:



...


 LaTeX code:
Matrix: \left(\begin{array}{r}
3 \\
3 \\
1 \\
1 \\
2 \\
2 \\
1 \\
1 \\
3
\end{array}\right)

 ------------------------------------------- 


s = -12, t = 19, s+t = 7 ; (j = 12, n = 7):

                    d : Hom^(-12)(6 wedges of g, k) --> Hom^(-12)(7 wedges of g, k)
                    
matrix shape: (2, 9)

 Matrix:



[2 0 1 0 2 4 2 4 0]
[0 3 3 2 4 1 3 3 0]


 LaTeX code:
Matrix: \left(\begin{array}{rrrrrrrrr}
2 & 0 & 1 & 0 & 2 & 4 & 2 & 4 & 0 \\
0 & 3 & 3 & 2 & 4 & 1 & 3 & 3 & 0
\end{array}\right)

 ------------------------------------------- 


s = -13, t = 20, s+t = 7 ; (j = 13, n = 7):

                    d : Hom^(-13)(6 wedges of g, k) --> Hom^(-13)(7 wedges of g, k)
                    
matrix shape: (3, 3)

 Matrix:



[4 1 2]
[4 2 2]
[2 2 4]


 LaTeX code:
Matrix: \left(\begin{array}{rrr}
4 & 1 & 2 \\
4 & 2 & 2 \\
2 & 2 & 4
\end{array}\right)

 ------------------------------------------- 




In [64]:
print_ker_im_bases(ID_ker_im_bases)


        Non trivial kernel and image bases:

        
s = -1, t = 3, s+t = 2 ; (j = 1, n = 2):

                    d : Hom^(-1)(1 wedges of g, k) --> Hom^(-1)(2 wedges of g, k)
                    
kernel basis:
    [
(1, 0, 0),
(0, 1, 0),
(0, 0, 1)
]

 ------------------------------------------- 


s = -2, t = 4, s+t = 2 ; (j = 2, n = 2):

                    d : Hom^(-2)(1 wedges of g, k) --> Hom^(-2)(2 wedges of g, k)
                    
image basis:
    [
(1, 0, 0),
(0, 1, 0),
(0, 0, 1)
]

 ------------------------------------------- 


s = -2, t = 5, s+t = 3 ; (j = 2, n = 3):

                    d : Hom^(-2)(2 wedges of g, k) --> Hom^(-2)(3 wedges of g, k)
                    
kernel basis:
    [
(1, 0, 0),
(0, 1, 0),
(0, 0, 1)
]

 ------------------------------------------- 


s = -3, t = 5, s+t = 2 ; (j = 3, n = 2):

                    d : Hom^(-3)(1 wedges of g, k) --> Hom^(-3)(2 wedges of g, k)
                    
image basis:
    [
(0, 1, 0, 0, 2, 1, 3, 4, 4),
(0, 0, 1,

In [68]:
print_H_s_t(ID_H_s_t_array)


        Non trivial cohomology H^(s,t) spaces:

        
(s,t) = (0,0): dim H^(0,0) = 1

H^(0,0) = Vector space quotient V/W of dimension 1 over Finite Field of size 5 where
V: Vector space of dimension 1 over Finite Field of size 5
W: Vector space of degree 1 and dimension 0 over Finite Field of size 5
Basis matrix:
[]
---

(s,t) = (-1,1): dim H^(-1,2) = 3

H^(-1,2) = Vector space quotient V/W of dimension 3 over Finite Field of size 5 where
V: Vector space of degree 3 and dimension 3 over Finite Field of size 5
Basis matrix:
[1 0 0]
[0 1 0]
[0 0 1]
W: Vector space of degree 3 and dimension 0 over Finite Field of size 5
Basis matrix:
[]
---

(s,t) = (-3,3): dim H^(-3,5) = 6

H^(-3,5) = Vector space quotient V/W of dimension 6 over Finite Field of size 5 where
V: Vector space of degree 9 and dimension 8 over Finite Field of size 5
Basis matrix:
[1 0 0 0 0 0 0 0 4]
[0 1 0 0 0 0 0 0 2]
[0 0 1 0 0 0 0 0 3]
[0 0 0 1 0 0 0 0 4]
[0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 1 0 0 2]
[0 0 0 0 0 0 1 0 3]
[0

In [69]:
print_H_s_t_bases(ID_H_s_t_array)


        Bases of non trivial cohomology H^(s,t) spaces:

        
(s,t) = (0,0): dim H^(0,0) = 1

basis in H^(0,0):
 ((1),)

lift of basis to the ambient space:
 [(1)]
---

(s,t) = (-1,1): dim H^(-1,2) = 3

basis in H^(-1,2):
 ((1, 0, 0), (0, 1, 0), (0, 0, 1))

lift of basis to the ambient space:
 [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
---

(s,t) = (-3,3): dim H^(-3,5) = 6

basis in H^(-3,5):
 ((1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0), (0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 1))

lift of basis to the ambient space:
 [(1, 0, 0, 0, 0, 0, 0, 0, 4), (0, 1, 0, 0, 0, 0, 0, 0, 2), (0, 0, 1, 0, 0, 0, 0, 0, 3), (0, 0, 0, 1, 0, 0, 0, 0, 4), (0, 0, 0, 0, 1, 0, 0, 0, 1), (0, 0, 0, 0, 0, 1, 0, 0, 2)]
---

(s,t) = (-4,4): dim H^(-4,6) = 3

basis in H^(-4,6):
 ((1, 0, 0), (0, 1, 0), (0, 0, 1))

lift of basis to the ambient space:
 [(1, 0, 0, 3, 3, 3, 0, 2, 1), (0, 1, 0, 0, 4, 4, 1, 3, 1), (0, 0, 1, 1, 0, 3, 0, 0, 2)]
---

(s,t) = (-4,4): dim H^(-4,7) = 3

basis in H^(-4,

# Cup products:

In [91]:
print_cup_products(g_ID, e_sort_ID, ID_H_s_t_array, ID_bases)


        Non-zero cup products from H^(s,t) spaces:

        
NOTE: We are not printing any H^(0) cup products.

We write vi^(s,t) for the i'th basis vector of H^(s,t).



v1^(-1,2) cup v1^(-3,5) = 2*v1^(-4,7) + 4*v2^(-4,7) + 1*v3^(-4,7)

v1^(-1,2) cup v2^(-3,5) = 1*v1^(-4,7) + 2*v2^(-4,7) + 3*v3^(-4,7)

v1^(-1,2) cup v3^(-3,5) = 4*v1^(-4,7) + 3*v2^(-4,7) + 2*v3^(-4,7)

v1^(-1,2) cup v4^(-3,5) = 3*v1^(-4,7) + 4*v2^(-4,7) + 1*v3^(-4,7)

v1^(-1,2) cup v5^(-3,5) = 3*v1^(-4,7) + 2*v2^(-4,7) + 4*v3^(-4,7)

v1^(-1,2) cup v6^(-3,5) = 1*v1^(-4,7) + 2*v2^(-4,7) + 4*v3^(-4,7)

v2^(-1,2) cup v1^(-3,5) = 2*v1^(-4,7) + 4*v2^(-4,7)

v2^(-1,2) cup v2^(-3,5) = 4*v1^(-4,7) + 1*v2^(-4,7)

v2^(-1,2) cup v3^(-3,5) = 1*v1^(-4,7) + 3*v2^(-4,7) + 4*v3^(-4,7)

v2^(-1,2) cup v4^(-3,5) = 3*v1^(-4,7) + 4*v2^(-4,7)

v2^(-1,2) cup v5^(-3,5) = 2*v1^(-4,7) + 1*v2^(-4,7)

v2^(-1,2) cup v6^(-3,5) = 4*v1^(-4,7) + 2*v2^(-4,7)

v3^(-1,2) cup v1^(-3,5) = 2*v1^(-4,7) + 4*v2^(-4,7) + 3*v3^(-4,7)

v3^(-1,2) cup v2^(-3,5) = 4


v1^(-3,5) cup v1^(-6,9) = 4*v1^(-9,14) + 1*v2^(-9,14) + 3*v4^(-9,14) + 4*v5^(-9,14) + 4*v6^(-9,14) + 4*v7^(-9,14)

v1^(-3,5) cup v2^(-6,9) = 3*v1^(-9,14) + 3*v2^(-9,14) + 2*v3^(-9,14) + 2*v4^(-9,14) + 2*v5^(-9,14) + 4*v6^(-9,14)

v1^(-3,5) cup v3^(-6,9) = 3*v1^(-9,14) + 3*v3^(-9,14) + 2*v4^(-9,14) + 3*v5^(-9,14)

v1^(-3,5) cup v4^(-6,9) = 2*v1^(-9,14) + 3*v2^(-9,14) + 1*v3^(-9,14) + 4*v4^(-9,14) + 4*v5^(-9,14)

v1^(-3,5) cup v5^(-6,9) = 1*v1^(-9,14) + 2*v2^(-9,14) + 4*v3^(-9,14) + 3*v4^(-9,14)

v1^(-3,5) cup v6^(-6,9) = 4*v1^(-9,14) + 1*v2^(-9,14) + 1*v3^(-9,14) + 1*v4^(-9,14) + 2*v5^(-9,14) + 3*v6^(-9,14) + 4*v7^(-9,14)

v1^(-3,5) cup v7^(-6,9) = 3*v1^(-9,14) + 1*v2^(-9,14) + 4*v3^(-9,14) + 3*v4^(-9,14) + 4*v6^(-9,14) + 1*v7^(-9,14)

v2^(-3,5) cup v1^(-6,9) = 2*v2^(-9,14) + 4*v3^(-9,14) + 2*v4^(-9,14) + 1*v5^(-9,14) + 4*v7^(-9,14)

v2^(-3,5) cup v2^(-6,9) = 3*v1^(-9,14) + 2*v2^(-9,14) + 2*v4^(-9,14) + 4*v5^(-9,14) + 4*v6^(-9,14) + 1*v7^(-9,14)

v2^(-3,5) cup v3^(-6,9) = 2*v1^(-9,14) 


v1^(-4,6) cup v1^(-7,11) = 2*v1^(-11,17) + 1*v2^(-11,17) + 2*v3^(-11,17)

v1^(-4,6) cup v2^(-7,11) = 3*v1^(-11,17) + 3*v2^(-11,17) + 1*v3^(-11,17)

v1^(-4,6) cup v3^(-7,11) = 2*v1^(-11,17)

v1^(-4,6) cup v4^(-7,11) = 3*v1^(-11,17) + 2*v2^(-11,17) + 3*v3^(-11,17)

v1^(-4,6) cup v5^(-7,11) = 2*v1^(-11,17)

v1^(-4,6) cup v6^(-7,11) = 2*v1^(-11,17) + 3*v2^(-11,17) + 3*v3^(-11,17)

v1^(-4,6) cup v7^(-7,11) = 3*v1^(-11,17) + 4*v3^(-11,17)

v1^(-4,6) cup v8^(-7,11) = 3*v1^(-11,17) + 2*v2^(-11,17) + 2*v3^(-11,17)

v1^(-4,6) cup v9^(-7,11) = 3*v1^(-11,17) + 4*v2^(-11,17)

v2^(-4,6) cup v1^(-7,11) = 1*v1^(-11,17) + 1*v3^(-11,17)

v2^(-4,6) cup v2^(-7,11) = 3*v1^(-11,17) + 4*v2^(-11,17)

v2^(-4,6) cup v3^(-7,11) = 1*v1^(-11,17) + 2*v2^(-11,17)

v2^(-4,6) cup v4^(-7,11) = 1*v1^(-11,17) + 3*v2^(-11,17) + 4*v3^(-11,17)

v2^(-4,6) cup v5^(-7,11) = 4*v1^(-11,17) + 1*v2^(-11,17) + 3*v3^(-11,17)

v2^(-4,6) cup v6^(-7,11) = 4*v1^(-11,17) + 4*v2^(-11,17) + 3*v3^(-11,17)

v2^(-4,6) cup v7^(-7,11) = 4*v1^(



v1^(-6,9) cup v2^(-6,9) = 4*v1^(-12,18) + 3*v2^(-12,18) + 4*v3^(-12,18) + 3*v5^(-12,18) + 3*v6^(-12,18)

v1^(-6,9) cup v3^(-6,9) = 3*v2^(-12,18) + 3*v3^(-12,18) + 3*v4^(-12,18) + 4*v5^(-12,18) + 1*v6^(-12,18)

v1^(-6,9) cup v4^(-6,9) = 1*v1^(-12,18) + 2*v2^(-12,18) + 2*v3^(-12,18) + 1*v4^(-12,18) + 1*v5^(-12,18)

v1^(-6,9) cup v5^(-6,9) = 2*v3^(-12,18) + 1*v4^(-12,18) + 3*v5^(-12,18) + 1*v6^(-12,18)

v1^(-6,9) cup v6^(-6,9) = 3*v1^(-12,18) + 1*v2^(-12,18) + 3*v3^(-12,18) + 3*v5^(-12,18) + 4*v6^(-12,18)

v1^(-6,9) cup v7^(-6,9) = 1*v1^(-12,18) + 4*v2^(-12,18) + 1*v3^(-12,18) + 3*v5^(-12,18) + 2*v6^(-12,18)

v2^(-6,9) cup v1^(-6,9) = 1*v1^(-12,18) + 2*v2^(-12,18) + 1*v3^(-12,18) + 2*v5^(-12,18) + 2*v6^(-12,18)

v2^(-6,9) cup v3^(-6,9) = 4*v1^(-12,18) + 3*v2^(-12,18) + 2*v3^(-12,18) + 3*v4^(-12,18) + 4*v5^(-12,18) + 2*v6^(-12,18)

v2^(-6,9) cup v4^(-6,9) = 4*v1^(-12,18) + 4*v2^(-12,18) + 1*v3^(-12,18) + 4*v4^(-12,18) + 1*v5^(-12,18) + 2*v6^(-12,18)

v2^(-6,9) cup v5^(-6,9) = 1*v1^(-12,1

In [92]:
print_cup_products_alt(g_ID, e_sort_ID, ID_H_s_t_array, ID_bases)


        Non-zero cup products from H^(s,t) spaces:

        
NOTE: We are not printing any H^(0) cup products.

We write vi^(s,t) for the i'th basis vector of H^(s,t).



v1^(-1,2) cup v1^(-3,5) = (2, 4, 1) in H^(-4,7)

v1^(-1,2) cup v2^(-3,5) = (1, 2, 3) in H^(-4,7)

v1^(-1,2) cup v3^(-3,5) = (4, 3, 2) in H^(-4,7)

v1^(-1,2) cup v4^(-3,5) = (3, 4, 1) in H^(-4,7)

v1^(-1,2) cup v5^(-3,5) = (3, 2, 4) in H^(-4,7)

v1^(-1,2) cup v6^(-3,5) = (1, 2, 4) in H^(-4,7)

v2^(-1,2) cup v1^(-3,5) = (2, 4, 0) in H^(-4,7)

v2^(-1,2) cup v2^(-3,5) = (4, 1, 0) in H^(-4,7)

v2^(-1,2) cup v3^(-3,5) = (1, 3, 4) in H^(-4,7)

v2^(-1,2) cup v4^(-3,5) = (3, 4, 0) in H^(-4,7)

v2^(-1,2) cup v5^(-3,5) = (2, 1, 0) in H^(-4,7)

v2^(-1,2) cup v6^(-3,5) = (4, 2, 0) in H^(-4,7)

v3^(-1,2) cup v1^(-3,5) = (2, 4, 3) in H^(-4,7)

v3^(-1,2) cup v2^(-3,5) = (4, 1, 2) in H^(-4,7)

v3^(-1,2) cup v3^(-3,5) = (2, 4, 1) in H^(-4,7)

v3^(-1,2) cup v4^(-3,5) = (4, 1, 3) in H^(-4,7)

v3^(-1,2) cup v5^(-3,5) = (4, 0, 4) in H^(-4


v1^(-4,6) cup v1^(-7,11) = (2, 1, 2) in H^(-11,17)

v1^(-4,6) cup v2^(-7,11) = (3, 3, 1) in H^(-11,17)

v1^(-4,6) cup v3^(-7,11) = (2, 0, 0) in H^(-11,17)

v1^(-4,6) cup v4^(-7,11) = (3, 2, 3) in H^(-11,17)

v1^(-4,6) cup v5^(-7,11) = (2, 0, 0) in H^(-11,17)

v1^(-4,6) cup v6^(-7,11) = (2, 3, 3) in H^(-11,17)

v1^(-4,6) cup v7^(-7,11) = (3, 0, 4) in H^(-11,17)

v1^(-4,6) cup v8^(-7,11) = (3, 2, 2) in H^(-11,17)

v1^(-4,6) cup v9^(-7,11) = (3, 4, 0) in H^(-11,17)

v2^(-4,6) cup v1^(-7,11) = (1, 0, 1) in H^(-11,17)

v2^(-4,6) cup v2^(-7,11) = (3, 4, 0) in H^(-11,17)

v2^(-4,6) cup v3^(-7,11) = (1, 2, 0) in H^(-11,17)

v2^(-4,6) cup v4^(-7,11) = (1, 3, 4) in H^(-11,17)

v2^(-4,6) cup v5^(-7,11) = (4, 1, 3) in H^(-11,17)

v2^(-4,6) cup v6^(-7,11) = (4, 4, 3) in H^(-11,17)

v2^(-4,6) cup v7^(-7,11) = (4, 1, 2) in H^(-11,17)

v2^(-4,6) cup v8^(-7,11) = (1, 1, 1) in H^(-11,17)

v2^(-4,6) cup v9^(-7,11) = (3, 1, 3) in H^(-11,17)

v3^(-4,6) cup v1^(-7,11) = (3, 2, 0) in H^(-11,17)

v3^(-4,6) c


v1^(-6,9) cup v2^(-9,14) = (1) in H^(-15,23)

v1^(-6,9) cup v3^(-9,14) = (1) in H^(-15,23)

v1^(-6,9) cup v4^(-9,14) = (3) in H^(-15,23)

v1^(-6,9) cup v5^(-9,14) = (2) in H^(-15,23)

v1^(-6,9) cup v6^(-9,14) = (4) in H^(-15,23)

v1^(-6,9) cup v7^(-9,14) = (4) in H^(-15,23)

v2^(-6,9) cup v1^(-9,14) = (3) in H^(-15,23)

v2^(-6,9) cup v2^(-9,14) = (1) in H^(-15,23)

v2^(-6,9) cup v3^(-9,14) = (4) in H^(-15,23)

v2^(-6,9) cup v4^(-9,14) = (1) in H^(-15,23)

v2^(-6,9) cup v5^(-9,14) = (1) in H^(-15,23)

v2^(-6,9) cup v6^(-9,14) = (4) in H^(-15,23)

v2^(-6,9) cup v7^(-9,14) = (2) in H^(-15,23)

v3^(-6,9) cup v1^(-9,14) = (1) in H^(-15,23)

v3^(-6,9) cup v2^(-9,14) = (2) in H^(-15,23)

v3^(-6,9) cup v3^(-9,14) = (4) in H^(-15,23)

v3^(-6,9) cup v6^(-9,14) = (3) in H^(-15,23)

v3^(-6,9) cup v7^(-9,14) = (3) in H^(-15,23)

v4^(-6,9) cup v1^(-9,14) = (1) in H^(-15,23)

v4^(-6,9) cup v4^(-9,14) = (1) in H^(-15,23)

v4^(-6,9) cup v5^(-9,14) = (1) in H^(-15,23)

v4^(-6,9) cup v6^(-9,14) = (3) in

# Final note

We can use inbuilt sage functions to double check that some of our cohomology calculations (without considering grading) are correct.

In [95]:
show("SL_3(Z_p)")
for ring in [QQ,ZZ,GF(2),GF(3),GF(5),GF(7),GF(11),GF(13)]:
    tmpL = LieAlgebra(ring, {
            ('x1','x6'): {'x2':-1},
            ( 'x1','x7'): {'x3':1},
            ('x1','x8'): {'x4':-1, 'x5':-1},
            ('x2','x7'): {'x4':-1},
            ('x3','x6'): {'x5':-1},
            ('x6','x7'): {'x8':-1},
        }, names='x1,x2,x3,x4,x5,x6,x7,x8')
    show(tmpL.cohomology())

'SL_3(Z_p)'

{0: Vector space of dimension 1 over Rational Field,
 1: Vector space of dimension 3 over Rational Field,
 2: Vector space of dimension 9 over Rational Field,
 3: Vector space of dimension 16 over Rational Field,
 4: Vector space of dimension 18 over Rational Field,
 5: Vector space of dimension 16 over Rational Field,
 6: Vector space of dimension 9 over Rational Field,
 7: Vector space of dimension 3 over Rational Field,
 8: Vector space of dimension 1 over Rational Field}

{0: Z,
 1: Z x Z x Z,
 2: Z^9,
 3: Z^16,
 4: Z^18 x C2 x C2,
 5: Z^16 x C2 x C2,
 6: Z^9,
 7: Z x Z x Z,
 8: Z}

{0: Vector space of dimension 1 over Finite Field of size 2,
 1: Vector space of dimension 3 over Finite Field of size 2,
 2: Vector space of dimension 9 over Finite Field of size 2,
 3: Vector space of dimension 18 over Finite Field of size 2,
 4: Vector space of dimension 22 over Finite Field of size 2,
 5: Vector space of dimension 18 over Finite Field of size 2,
 6: Vector space of dimension 9 over Finite Field of size 2,
 7: Vector space of dimension 3 over Finite Field of size 2,
 8: Vector space of dimension 1 over Finite Field of size 2}

{0: Vector space of dimension 1 over Finite Field of size 3,
 1: Vector space of dimension 3 over Finite Field of size 3,
 2: Vector space of dimension 9 over Finite Field of size 3,
 3: Vector space of dimension 16 over Finite Field of size 3,
 4: Vector space of dimension 18 over Finite Field of size 3,
 5: Vector space of dimension 16 over Finite Field of size 3,
 6: Vector space of dimension 9 over Finite Field of size 3,
 7: Vector space of dimension 3 over Finite Field of size 3,
 8: Vector space of dimension 1 over Finite Field of size 3}

{0: Vector space of dimension 1 over Finite Field of size 5,
 1: Vector space of dimension 3 over Finite Field of size 5,
 2: Vector space of dimension 9 over Finite Field of size 5,
 3: Vector space of dimension 16 over Finite Field of size 5,
 4: Vector space of dimension 18 over Finite Field of size 5,
 5: Vector space of dimension 16 over Finite Field of size 5,
 6: Vector space of dimension 9 over Finite Field of size 5,
 7: Vector space of dimension 3 over Finite Field of size 5,
 8: Vector space of dimension 1 over Finite Field of size 5}

{0: Vector space of dimension 1 over Finite Field of size 7,
 1: Vector space of dimension 3 over Finite Field of size 7,
 2: Vector space of dimension 9 over Finite Field of size 7,
 3: Vector space of dimension 16 over Finite Field of size 7,
 4: Vector space of dimension 18 over Finite Field of size 7,
 5: Vector space of dimension 16 over Finite Field of size 7,
 6: Vector space of dimension 9 over Finite Field of size 7,
 7: Vector space of dimension 3 over Finite Field of size 7,
 8: Vector space of dimension 1 over Finite Field of size 7}

{0: Vector space of dimension 1 over Finite Field of size 11,
 1: Vector space of dimension 3 over Finite Field of size 11,
 2: Vector space of dimension 9 over Finite Field of size 11,
 3: Vector space of dimension 16 over Finite Field of size 11,
 4: Vector space of dimension 18 over Finite Field of size 11,
 5: Vector space of dimension 16 over Finite Field of size 11,
 6: Vector space of dimension 9 over Finite Field of size 11,
 7: Vector space of dimension 3 over Finite Field of size 11,
 8: Vector space of dimension 1 over Finite Field of size 11}

{0: Vector space of dimension 1 over Finite Field of size 13,
 1: Vector space of dimension 3 over Finite Field of size 13,
 2: Vector space of dimension 9 over Finite Field of size 13,
 3: Vector space of dimension 16 over Finite Field of size 13,
 4: Vector space of dimension 18 over Finite Field of size 13,
 5: Vector space of dimension 16 over Finite Field of size 13,
 6: Vector space of dimension 9 over Finite Field of size 13,
 7: Vector space of dimension 3 over Finite Field of size 13,
 8: Vector space of dimension 1 over Finite Field of size 13}